In [25]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Подготовительная часть задания

In [26]:
# Загрузить ДФ
#path = '/Users/vladislavkravchenko/Desktop/WORK_IT/1.Elbrus_Bootcamp/2.Phase0/4.Thursday/rent_real_estate/_data.csv'
df = pd.read_csv('_data.csv', header=0)
pd.set_option('display.max_columns', None)

In [27]:
# Выбор рабочих колонок, анализ которых позволяет провести графическую аналитику
df.rename(columns={'ID  объявления': 'ID объявления'}, inplace=True)
wdf = df[['Unnamed: 0',
    'ID объявления',  
   'Количество комнат', 
   'Метро', 
   'Площадь, м2', 
   'Дом', 
   'Парковка', 
   'Цена', 
   'Ремонт', 
   'Балкон', 
   'Окна', 
   'Санузел', 
   'Можно с детьми/животными',  
   'Высота потолков, м', 
   'Лифт', 
   'Мусоропровод']].copy()

In [28]:
# Замена нулевых значений
wdf.fillna(0, inplace=True)
wdf.isnull().sum()

Unnamed: 0                  0
ID объявления               0
Количество комнат           0
Метро                       0
Площадь, м2                 0
Дом                         0
Парковка                    0
Цена                        0
Ремонт                      0
Балкон                      0
Окна                        0
Санузел                     0
Можно с детьми/животными    0
Высота потолков, м          0
Лифт                        0
Мусоропровод                0
dtype: int64

In [29]:
# Шэйп ДФ для сверки результатов
wdf.shape

(23368, 16)

## Основная часть задания

### Колонка: Количество комнат. 
Задача: привести к типу int()

In [30]:
# Сплит по знаку запятой и взять только числа.
wdf = wdf.copy()
wdf['Количество комнат'] = wdf['Количество комнат'].apply(lambda x: x.split(',')[0] if type(x) != int else x)

In [31]:
wdf['Количество комнат'].value_counts()

Количество комнат
2    8466
1    7931
3    4262
4    1163
0    1041
5     367
6     138
Name: count, dtype: int64

In [32]:
# Проверка, что все значения численные
all(wdf['Количество комнат'].to_list()) == int()

True

### Колонка: Метро. 
Задача: оставить только название станции

In [33]:
wdf['Метро'] = wdf['Метро'].apply(lambda x: x.split() if type(x) == str else x)
wdf['Метро'] = wdf['Метро'].apply(lambda x: x[1] if type(x) == list else x)
wdf['Метро'].value_counts()


Метро
0              1315
Проспект        644
Улица           409
Селигерская     393
Щелковская      313
               ... 
Нахабино          1
Лужники           1
Яхромская         1
Румянцево         1
Волковская        1
Name: count, Length: 309, dtype: int64

### Колонка: Площадь, м2. 
Задача: оставить только первое значение = общая площадь, последнее значение = площадь кухни

In [11]:
wdf['Площадь, м2'] = wdf['Площадь, м2'].apply(lambda x: x.split('/') if type(x) == str else x)
wdf['Площадь, м2'] = wdf['Площадь, м2'].apply(lambda x: x[:] if len(x) == 2 else [x[0],x[-1]])
wdf['Площадь, м2'].value_counts()

Площадь, м2
[40.0, 10.0]      371
[45.0, 6.0]       344
[38.0, 8.0]       216
[38.0, 10.0]      198
[40.0, 9.0]       193
                 ... 
[146.0, 12.0]       1
[165.0, 116.0]      1
[158.6, 12.0]       1
[202.2, 202.2]      1
[205.0, 17.0]       1
Name: count, Length: 5372, dtype: int64

### Колонка: Дом. 
Задача: привести к списку вида [этаж, количество этажей, тип строения]

In [34]:
# Разделим по запятой
wdf['Дом'] = wdf['Дом'].apply(lambda x: x.split() if type(x) == str else [x])
# Уберем запятую в первом значении каждого списка
wdf['Дом'] = wdf['Дом'].apply(lambda x: [x[0].replace(',',''), x[1]] if len(x) >1 and type(x[0]) == str else x)
# Приведем данные к виду [x/x, text]
wdf['Дом'] = wdf['Дом'].apply(lambda x: [x[0], 0] if len(x) == 1 else x)
# Привед данные к виду [этаж, количество этажей, тип строения]
wdf['Дом'] = wdf['Дом'].apply(lambda x: [*x[0].split('/'), x[1]])
# Приведем данные этажа и количества этажей к числовому типу
wdf['Дом'] = wdf['Дом'].apply(lambda x: [int(x[0]), int(x[1]), x[2]])

### Колонка: Парковка.

In [35]:
# Данные по парковке чистые, не требуют преобразования
wdf['Парковка'].value_counts()

Парковка
0                 13417
наземная           6043
подземная          2772
открытая           1017
многоуровневая      118
на крыше              1
Name: count, dtype: int64

### Колонка: Ремонт.

In [36]:
# Данные по ремонту чистые, не требуют преобразования
wdf['Ремонт'].value_counts()

Ремонт
Косметический    8499
Евроремонт       8470
Дизайнерский     3474
0                2755
Без ремонта       170
Name: count, dtype: int64

### Колонка: Балкон. 
Задача: Привести данные к виду [num1 , num2], где num1 = Балкон, num2 = Лоджия

In [37]:
# Сплит данных по запятой
wdf['Балкон'] = wdf['Балкон'].apply(lambda x: x.split(', ') if type(x) == str else [x, x])
# Приведем данные к виду [num1, num2]
wdf['Балкон'] = wdf['Балкон'].apply(lambda x: [x[0], 0] if len(x) == 1 else x)
# Вытащим числовое значение в данных по балконам и приведем к числовому типу
wdf['Балкон'] = wdf['Балкон'].apply(lambda x: [int(x[0][-2]), x[1]] if type(x[0]) == str else x)
# Вытащим числовое значение в данных по лоджиям и приведем к числовому типу
wdf['Балкон'] = wdf['Балкон'].apply(lambda x: [x[0], int(x[1][-2])] if type(x[1]) == str else x)
wdf['Балкон'].value_counts()

Балкон
[1, 0]    13435
[0, 0]     7978
[2, 0]     1042
[1, 1]      716
[3, 0]      100
[2, 2]       25
[1, 2]       24
[2, 1]       20
[4, 0]       11
[3, 1]        5
[1, 3]        5
[2, 3]        3
[1, 4]        2
[3, 3]        1
[4, 4]        1
Name: count, dtype: int64

### Колонка: Окна

In [38]:
# Данные по окнам чистые, не требуют обработки
wdf['Окна'].value_counts()

Окна
Во двор            10870
0                   6613
На улицу и двор     3295
На улицу            2590
Name: count, dtype: int64

### Колонка: Санузел. 
Задача: Привести данные к виду [num1 , num2], где num1 = Совмещенный, num2 = Раздельный

In [39]:
# Сплит данных по запятой
wdf['Санузел'] = wdf['Санузел'].apply(lambda x: x.split(', ') if type(x) == str else [x, x])
# Приведем данные к виду [num1, num2]
wdf['Санузел'] = wdf['Санузел'].apply(lambda x: [x[0], 0] if len(x) == 1 else x)
# Вытащим числовое значение в данных по балконам и приведем к числовому типу
wdf['Санузел'] = wdf['Санузел'].apply(lambda x: [int(x[0][-2]), x[1]] if type(x[0]) == str else x)
# Вытащим числовое значение в данных по лоджиям и приведем к числовому типу
wdf['Санузел'] = wdf['Санузел'].apply(lambda x: [x[0], int(x[1][-2])] if type(x[1]) == str else x)
wdf['Санузел'].value_counts()

Санузел
[1, 0]    17236
[0, 0]     2672
[2, 0]     1971
[1, 1]      812
[3, 0]      293
[2, 1]      188
[4, 0]       92
[1, 2]       30
[3, 1]       27
[2, 2]       25
[3, 3]        6
[4, 1]        6
[4, 2]        4
[2, 3]        2
[1, 3]        2
[2, 4]        1
[3, 2]        1
Name: count, dtype: int64

### Колонка: Можно с детьми/животными. 

In [40]:
# Данные по лояльности к существам с потенциальным риском нанесения материального ущерба чистые, не требуют обработки
wdf['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       10134
Можно с детьми, Можно с животными     6899
0                                     6096
Можно с животными                      239
Name: count, dtype: int64

### Колонка: Высота потолков, м. 
Задача: Убрать выбросы

In [41]:
# Определим среднее значение высоты потолка
wdf[wdf['Высота потолков, м'].apply(lambda x: 2 < x < 4)]['Высота потолков, м'].mean()
# Получили значение 2.7
# Заменим выбросы на среднее значение высоты потолка
wdf['Высота потолков, м'] = wdf['Высота потолков, м'].apply(lambda x: 2.7 if x > 10 else x)
wdf['Высота потолков, м'].value_counts()

Высота потолков, м
0.00    12162
2.64     4467
3.00     1322
2.70     1058
2.48      676
        ...  
3.41        1
3.24        1
3.21        1
1.20        1
3.02        1
Name: count, Length: 85, dtype: int64

### Колонка: Лифт. 
Задача: Убрать выбросы

In [42]:
# Сплит данных по запятой
wdf['Лифт'] = wdf['Лифт'].apply(lambda x: x.split(', ') if type(x) == str else [x, x])
# Приведем данные к виду [num1, num2]
wdf['Лифт'] = wdf['Лифт'].apply(lambda x: [x[0], 0] if len(x) == 1 else x)
# Вытащим числовое значение в данных по балконам и приведем к числовому типу
wdf['Лифт'] = wdf['Лифт'].apply(lambda x: [int(x[0][-2]), x[1]] if type(x[0]) == str else x)
# Вытащим числовое значение в данных по лоджиям и приведем к числовому типу
wdf['Лифт'] = wdf['Лифт'].apply(lambda x: [x[0], int(x[1][-2])] if type(x[1]) == str else x)
wdf['Лифт'].value_counts()

Лифт
[1, 0]    6006
[0, 0]    5504
[2, 0]    4346
[1, 1]    3962
[2, 1]    1224
[2, 2]     654
[3, 0]     651
[4, 0]     371
[3, 1]     187
[1, 2]     186
[4, 2]      65
[4, 1]      64
[3, 3]      45
[3, 2]      44
[4, 4]      28
[4, 3]       7
[6, 0]       7
[1, 3]       6
[2, 3]       3
[5, 1]       1
[8, 8]       1
[7, 0]       1
[1, 4]       1
[5, 3]       1
[2, 4]       1
[8, 0]       1
[5, 0]       1
Name: count, dtype: int64

### Колонка: Мусоропровод. 

In [43]:
# Данные по мусоропроводам чистые, не требуют обработки
wdf['Мусоропровод'].value_counts()

Мусоропровод
Да     10897
0      10522
Нет     1949
Name: count, dtype: int64

## Финал. 
### Вид датафрейма с обработанными данными

In [44]:
wdf.head(3)
# Описание изменений. 
# Note: колонки без изменений помечены знакосочетанием -/-
# Column: Unnamed: 0. -/-
# Column: ID объявления. -/-
# Column: Количество комнат. Приведено к типу int()
# Column: Метро. Приведено к виду "Название метро"
# Column: Площадь, м2. Приведено к виду ['Общая площадь', 'Площадь кухни']
# Column: Дом. Приведено к виду ['Этаж', 'Количество этажей', 'Тип строения']
# Column: Парковка. -/-
# Column: Ремонт. -/-
# Column: Балкон. Приведено к виду ['Количество балконов', 'Количество лоджий']
# Column: Окна. -/-
# Column: Санузел. Приведено к виду ['Совмещенный', 'Раздельный']
# Column: Можно с детьми/животными. -/-
# Column: Высота потолков, м. Убраны выбросы
# Column: Лифт. Приведено к виду ['Пассажирский', 'Грузовой']
# Column: Мусоропровод. -/-

,Unnamed: 0,ID объявления,Количество комнат,Метро,"Площадь, м2",Дом,Парковка,Цена,Ремонт,Балкон,Окна,Санузел,Можно с детьми/животными,"Высота потолков, м",Лифт,Мусоропровод
0,0,271271157,4,Смоленская,200.0/20.0,"[5, 16, Монолитный]",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,"[0, 0]",0,"[0, 0]","Можно с детьми, Можно с животными",3.0,"[4, 1]",Да
1,1,271634126,4,Смоленская,198.0/95.0/18.0,"[5, 16, Монолитно-кирпичный]",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,"[0, 0]",На улицу и двор,"[2, 1]",Можно с детьми,3.5,"[1, 1]",Нет
2,2,271173086,4,Смоленская,200.0/116.0/4.0,"[5, 16, 0]",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,"[0, 0]",На улицу и двор,"[3, 0]",Можно с детьми,3.2,"[1, 0]",0


Формирование финального датафрейма, удобного для отображения графической аналитики. \
Колонки со значениями типа list() разбиты на дополнительные колонки

In [46]:
#Обработка колонки Цены. 
fdf = pd.DataFrame() # fdf = final data frame
fdf['Unnamed: 0'] = wdf['Unnamed: 0']
fdf['ID объявления'] = wdf['ID объявления']
fdf['Количество комнат'] = wdf['Количество комнат']
fdf['Метро'] = wdf['Метро']
fdf['Общая площадь, м2'] = wdf['Площадь, м2'].apply(lambda x: x[0])
fdf['Площадь кухни, м2'] = wdf['Площадь, м2'].apply(lambda x: x[1])
fdf['Этаж'] = wdf['Дом'].apply(lambda x: x[0])
fdf['Этажей в доме'] = wdf['Дом'].apply(lambda x: x[1])
fdf['Тип строения'] = wdf['Дом'].apply(lambda x: x[2])
fdf['Парковка'] = wdf['Парковка']
# Вставить ! данные по ценам !
fdf['Ремонт'] = wdf['Ремонт']
fdf['Балкон'] = wdf['Балкон'].apply(lambda x: x[0])
fdf['Лоджия'] = wdf['Балкон'].apply(lambda x: x[1])
fdf['Окна'] = wdf['Окна']
fdf['Санузел раздельный'] = wdf['Санузел'].apply(lambda x: x[0])
fdf['Санузел совмещенный'] = wdf['Санузел'].apply(lambda x: x[1])
fdf['Можно с детьми/животными'] = wdf['Можно с детьми/животными']
fdf['Высота потолков, м'] = wdf['Высота потолков, м']
fdf['Лифты пасс.'] = wdf['Лифт'].apply(lambda x: x[0])
fdf['Лифты груз.'] = wdf['Лифт'].apply(lambda x: x[1])
fdf['Мусоропровод'] = wdf['Мусоропровод']

fdf

,Квартплата,Залог,комм,Срок,Предоплата,Unnamed: 0,ID объявления,Количество комнат,Метро,"Общая площадь, м2","Площадь кухни, м2",Этаж,Этажей в доме,Тип строения,Парковка,Ремонт,Балкон,Лоджия,Окна,Санузел раздельный,Санузел совмещенный,Можно с детьми/животными,"Высота потолков, м",Лифты пасс.,Лифты груз.,Мусоропровод
0,500000.0,500000,0,Срок аренды - Длительный,1,0,271271157,4,Смоленская,2,0,5,16,Монолитный,подземная,Дизайнерский,0,0,0,0,0,"Можно с детьми, Можно с животными",3.00,4,1,Да
1,500000.0,500000,0,Срок аренды - Длительный,1,1,271634126,4,Смоленская,1,9,5,16,Монолитно-кирпичный,подземная,Дизайнерский,0,0,На улицу и двор,2,1,Можно с детьми,3.50,1,1,Нет
2,500000.0,500000,[],Срок аренды - Длительный,1,2,271173086,4,Смоленская,2,0,5,16,0,подземная,Евроремонт,0,0,На улицу и двор,3,0,Можно с детьми,3.20,1,0,0
3,400000.0,400000,[],Срок аренды - Длительный,1,3,272197456,4,Смоленская,1,7,5,6,0,подземная,Евроремонт,0,0,На улицу и двор,3,0,Можно с животными,3.20,1,0,0
4,225000.0,225000,0,Срок аренды - Длительный,1,4,273614615,2,Арбатская,5,8,12,26,Панельный,0,Евроремонт,0,0,На улицу и двор,2,0,0,3.90,1,1,Да
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,42000.0,42000,0,Срок аренды - Длительный,1,23363,215565511,0,Говорово,3,5,10,14,0,0,Евроремонт,1,0,0,1,0,0,0.00,1,1,0
23364,45000.0,45000,0,Срок аренды - Длительный,1,23364,274654844,1,Солнцево,3,8,5,18,Монолитный,0,Евроремонт,1,0,Во двор,1,0,0,0.00,1,1,0
23365,50000.0,50000,0,Срок аренды - Длительный,1,23365,268679909,2,Солнцево,4,3,5,5,Кирпичный,0,Дизайнерский,1,0,На улицу и двор,1,0,Можно с детьми,0.00,0,0,0
23366,55000.0,50000,0,Срок аренды - Длительный,2,23366,274807525,2,Солнцево,5,2,8,23,Монолитный,наземная,Евроремонт,1,0,Во двор,1,1,0,2.65,3,0,Да


In [50]:


#Преобразуем колонку Цены в списки
counts = wdf.copy()
counts['Цена'] = wdf['Цена'].apply(lambda x: x.split(','))

#Теперь функция для обработки каждого элемента (i) в списке Series (х)

def price(x):
    communal = []      #коммуналка
    prepayment = []    #предоплата
    bail = []          #залог
    rent = []          #квартплата
    rental = []        #срок
    
    for i in x:   #берем i-ый элемент списка Цены (х)

        
        if 'За месяц' in i:                    #здесь две проверки на наличие валют и перевод в рубли. так же ищем в 
            if '$' in i:                       # строке только числовые данные и сохраняем их в Квартплату
                numbers = (re.findall(r'\d+\.\d+|\d+', i))[0]
                rent = float(numbers[0]) * 97
            elif '€' in i:
                numbers = (re.findall(r'\d+\.\d+|\d+', i))[0]
                rent = float(numbers[0]) * 106
            else:
                numbers = re.findall(r'\d+\.\d+|\d+', i) 
                rent = float(numbers[0])


        
        elif 'Залог' in i or 'залог' in i:      #здесь две проверки на наличие валют и перевод в рубли. так же ищем в 
            if '$' in i:                        #строке только числовые данные и сохраняем их в Залог
                numbers = (re.findall(r'\b\d+\b', i))[0]
                bail = int(numbers[0]) * 97
            elif '€' in i:
                numbers = (re.findall(r'\b\d+\b', i))[0]
                bail = int(numbers[0]) * 106
            else:
                numbers = re.findall(r'\b\d+\b', i) 
                bail = int(numbers[0])

        
        elif 'комм' in i or 'Комм' in i:       #здесь две проверки на необходисомть платить за комуналку и сами цены 
            if 'включены' in i:
                communal = 0
            else:
                numbers = (re.findall(r'\b\d+\b', i))[0]
                communal= float(numbers)                
                
        
        elif 'Срок' in i or 'срок' in i:      #здесь ищем и сохраняем срок аренды
            rental.append(i)

        
        elif 'Предоплата' in i:               #здесь ищем и сохраняем на какое количество месяцев необходимо внести Предоплату
            numbers = re.findall(r'\d+', i)
            prepayment = int(numbers[0])

         
    return pd.Series({
        'Квартплата': rent if rent else 0,
        'Залог': bail if bail else 0,
        'комм': communal if communal is not None else None,
        'Срок': ', '.join(rental) if rental else 0,
        'Предоплата': prepayment if prepayment else 0
    })
        
#Теперь создаем новые колонки, данные для которых берем из Цена
fdf[['Квартплата', 'Залог', 'комм', 'Срок', 'Предоплата']] = counts['Цена'].apply(price)

#Теперь делаю 0 там где коммуналку включена в стоимость аренды, а остальные пустые значения заменяю средними.
fdf['комм'] = fdf['комм'].apply(lambda x: np.nan if x == [] else x)
fdf['комм'].fillna(fdf['комм'].mean(), inplace=True)

#Немного округляю среднее
fdf['комм'] = fdf['комм'].round(3)

/tmp/ipykernel_68341/1871633310.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fdf['комм'].fillna(fdf['комм'].mean(), inplace=True)


In [51]:
fdf

,Квартплата,Залог,комм,Срок,Предоплата,Unnamed: 0,ID объявления,Количество комнат,Метро,"Общая площадь, м2","Площадь кухни, м2",Этаж,Этажей в доме,Тип строения,Парковка,Ремонт,Балкон,Лоджия,Окна,Санузел раздельный,Санузел совмещенный,Можно с детьми/животными,"Высота потолков, м",Лифты пасс.,Лифты груз.,Мусоропровод
0,500000.0,500000,0.000,Срок аренды - Длительный,1,0,271271157,4,Смоленская,2,0,5,16,Монолитный,подземная,Дизайнерский,0,0,0,0,0,"Можно с детьми, Можно с животными",3.00,4,1,Да
1,500000.0,500000,0.000,Срок аренды - Длительный,1,1,271634126,4,Смоленская,1,9,5,16,Монолитно-кирпичный,подземная,Дизайнерский,0,0,На улицу и двор,2,1,Можно с детьми,3.50,1,1,Нет
2,500000.0,500000,642.191,Срок аренды - Длительный,1,2,271173086,4,Смоленская,2,0,5,16,0,подземная,Евроремонт,0,0,На улицу и двор,3,0,Можно с детьми,3.20,1,0,0
3,400000.0,400000,642.191,Срок аренды - Длительный,1,3,272197456,4,Смоленская,1,7,5,6,0,подземная,Евроремонт,0,0,На улицу и двор,3,0,Можно с животными,3.20,1,0,0
4,225000.0,225000,0.000,Срок аренды - Длительный,1,4,273614615,2,Арбатская,5,8,12,26,Панельный,0,Евроремонт,0,0,На улицу и двор,2,0,0,3.90,1,1,Да
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,42000.0,42000,0.000,Срок аренды - Длительный,1,23363,215565511,0,Говорово,3,5,10,14,0,0,Евроремонт,1,0,0,1,0,0,0.00,1,1,0
23364,45000.0,45000,0.000,Срок аренды - Длительный,1,23364,274654844,1,Солнцево,3,8,5,18,Монолитный,0,Евроремонт,1,0,Во двор,1,0,0,0.00,1,1,0
23365,50000.0,50000,0.000,Срок аренды - Длительный,1,23365,268679909,2,Солнцево,4,3,5,5,Кирпичный,0,Дизайнерский,1,0,На улицу и двор,1,0,Можно с детьми,0.00,0,0,0
23366,55000.0,50000,0.000,Срок аренды - Длительный,2,23366,274807525,2,Солнцево,5,2,8,23,Монолитный,наземная,Евроремонт,1,0,Во двор,1,1,0,2.65,3,0,Да
